In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [62]:
mmseqs_df = pd.read_csv('mmseqs_cluster.tsv', sep="\t")
foldseek_df = pd.read_csv('foldseek_result_cluster.tsv', sep="\t")

In [63]:
mmseqs_df.columns = ['mmseqs_cluster', 'unclustered']

foldseek_df.columns = ['foldseek_cluster', 'unclustered']


In [64]:
pattern = r'UniRef100_([A-Za-z0-9]+)\|'
mmseqs_df['mmseqs_cluster'] = mmseqs_df['mmseqs_cluster'].str.extract(pattern)
mmseqs_df['unclustered'] = mmseqs_df['unclustered'].str.extract(pattern)


display(mmseqs_df.head())


,mmseqs_cluster,unclustered
0,A0A174B8I9,A0A174B8I9
1,A0A1E2RWF6,A0A1E2RWF6
2,A0A1V6HF93,A0A1V6HF93
3,A0A380MZP3,A0A380MZP3
4,A0A428I3D7,A0A428I3D7


In [65]:
foldseek_df['foldseek_cluster'] = foldseek_df['foldseek_cluster'].apply(lambda x: x.split('-')[1])
foldseek_df['unclustered'] = foldseek_df['unclustered'].apply(lambda x: x.split('-')[1])


display(foldseek_df.head())

,foldseek_cluster,unclustered
0,A0A078MK14,A0A4V2DZ80
1,A0A078MK14,A0A2R2W5C2
2,A0A0A2VBZ4,A0A0A2VBZ4
3,A0A0A2VBZ4,A0A1G8S2X6
4,A0A0B7MQS5,A0A0B7MQS5


In [51]:
mmseqs_df.to_csv('mmseqs__cleaned_cluster.tsv', sep="\t", index=False)
foldseek_df.to_csv('foldseek__cleaned_cluster.tsv', sep="\t", index=False)


In [66]:
# Group by the representative sequence and aggregate the clustered sequences into lists
cluster_dict = foldseek_df.groupby('foldseek_cluster')['unclustered'].apply(list).to_dict()

# Preview the first few items in the dictionary to verify the structure
list(cluster_dict.items())[:5]

[('A0A078MK14', ['A0A4V2DZ80', 'A0A2R2W5C2']),
 ('A0A0A2VBZ4', ['A0A0A2VBZ4', 'A0A1G8S2X6']),
 ('A0A0B7MQS5', ['A0A0B7MQS5']),
 ('A0A0D0RVH7', ['A0A0D0RVH7', 'A0A098EI80', 'A0A2S5D4V8']),
 ('A0A0G1MTH1', ['A0A0G1MTH1'])]

In [69]:
mmseqs_values = mmseqs_df['mmseqs_cluster'].tolist()

print(mmseqs_values)

['A0A174B8I9', 'A0A1E2RWF6', 'A0A1V6HF93', 'A0A380MZP3', 'A0A428I3D7', 'R5RW36', 'R5RW36', 'A0A285NMQ5', 'A0A285NMQ5', 'A0A285NMQ5', 'A0A7J0AAJ5', 'A0A916HN28', 'A0A916HN28', 'A0A916HN28', 'A0A916HN28', 'A0A916HN28', 'A0A916HN28', 'A0A916HN28', 'A0A916HN28', 'A0A916HN28', 'A0A916HN28', 'A0A916HN28', 'A0A916HN28', 'A0A916HN28', 'A0A916HN28', 'A0A916HN28', 'A0A916HN28', 'A0A916HN28', 'A0A916HN28', 'A0A0J6VLZ1', 'A0A0J6VLZ1', 'A0A1C6G745', 'A0A0J1I6K8', 'A0A1V4SDX7', 'A0A3S5AQD8', 'A0A3S5AQD8', 'A0A3S5AQD8', 'A0A3S5AQD8', 'A0A3S5AQD8', 'A0A6M1ZB96', 'A0A248JXY1', 'A0A433XPS9', 'A0A433XPS9', 'A0A1C5ML14', 'A0A1C5YPL9', 'A0A1V4WZV6', 'A0A1V6JEK4', 'A0A246KEF4', 'A0A246KEF4', 'A0A246KEF4', 'A0A246KEF4', 'A0A6N2SLV1', 'A0A977II70', 'R5STP1', 'A0A7U9MND5', 'A0A7U9MND5', 'R6SGY6', 'A0A1I6KYN6', 'A0A841RQA6', 'A0A841RQA6', 'A0A841RQA6', 'A0A841RQA6', 'A0A841RQA6', 'A0A841RQA6', 'A0A841RQA6', 'A0A1U7M8N9', 'A0A379DHS9', 'A0A379DHS9', 'A0A6N7B1P8', 'C3JB20', 'A0A0V8JNZ3', 'A0A0V8JNZ3', 'A0A0V8JNZ3

In [71]:
foldseek_list = []
hits = []



for value in mmseqs_values:

    for key, content in cluster_dict.items():
        if value in content:
            foldseek_list.append(key)
            # hits.append(value)
            
print(len(foldseek_list))


1879


In [55]:
no_match = [x for x in mmseqs_values if x not in hits]

print(hits)

[]


In [56]:
print(len(foldseek_list))

0
